# Topic Modeling with LDA

This is like unsupervised learning (ie. clustering) for Natural Language Processing.

# Loading Data

In [1]:
# Importing modules
import pandas as pd

# Read data into papers
papers = pd.read_csv('papers.csv')

# Print head
papers.head()

,id,year,title,event_type,pdf_name,abstract,paper_text
0,1,1987,Self-Organization of Associative Database and ...,NaN,1-self-organization-of-associative-database-an...,Abstract Missing,767\n\nSELF-ORGANIZATION OF ASSOCIATIVE DATABA...
1,10,1987,A Mean Field Theory of Layer IV of Visual Cort...,NaN,10-a-mean-field-theory-of-layer-iv-of-visual-c...,Abstract Missing,683\n\nA MEAN FIELD THEORY OF LAYER IV OF VISU...
2,100,1988,Storing Covariance by the Associative Long-Ter...,NaN,100-storing-covariance-by-the-associative-long...,Abstract Missing,394\n\nSTORING COVARIANCE BY THE ASSOCIATIVE\n...
3,1000,1994,Bayesian Query Construction for Neural Network...,NaN,1000-bayesian-query-construction-for-neural-ne...,Abstract Missing,Bayesian Query Construction for Neural\nNetwor...
4,1001,1994,"Neural Network Ensembles, Cross Validation, an...",NaN,1001-neural-network-ensembles-cross-validation...,Abstract Missing,"Neural Network Ensembles, Cross\nValidation, a..."


# Data Cleansing

Since the goal of this analysis is to perform topic modeling, let's focus only on the text data from each paper, and drop other metadata columns. Also, for the demonstration, we'll only look at 100 papers

In [2]:
# Remove the columns
papers = papers.drop(columns=['id', 'event_type', 'pdf_name'], axis=1).sample(100)

# Print out the first rows of papers
papers.head()

,year,title,abstract,paper_text
1024,2000,Programmable Reinforcement Learning Agents,Abstract Missing,Programmable Reinforcement Learning Agents\n\n...
2116,2005,Learning Depth from Single Monocular Images,Abstract Missing,Learning Depth from Single Monocular Images\n\...
357,1996,Estimating Equivalent Kernels for Neural Netwo...,Abstract Missing,Estimating Equivalent Kernels For Neural\nNetw...
614,1998,Probabilistic Visualisation of High-Dimensiona...,Abstract Missing,Probabilistic Visualisation of\nHigh-dimension...
6092,2016,Depth from a Single Image by Harmonizing Overc...,A single color image can contain many cues inf...,Depth from a Single Image by Harmonizing\nOver...
